In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from my_helper import *
from implementations import *

In [4]:
# pre-process

num_feature = tX.shape[1]
col_avg = np.zeros(num_feature)
col_std = np.zeros(num_feature)

for i in range(num_feature):
    cur_col = tX[:, i]
    if i == num_feature-1:
        good_data = np.where(cur_col!=0)
    else:
        good_data = np.where(cur_col!=-999)
    col_avg[i] = np.mean(cur_col[good_data])
    col_std[i] = np.std(cur_col[good_data])
    
col_avg = col_avg[np.newaxis, :]
col_std = col_std[np.newaxis, :]


bad_data = np.where(tX==-999)
bad_col_data = np.where(tX[:, -1]==0)

tX = (tX-col_avg) / col_std
tX[bad_data] = 0
tX[:, -1][bad_col_data] = 0

# std = np.std(tX, axis=0)
# tX = tX / std

In [5]:
# required implementations

# max_iters = 300
# least square
# gamma = 0.0000001
# logistic
# gamma = 1
# batch_size = 32
# lambda_ = 0.1
# initial_w = np.random.rand(tX.shape[1]) * 2 - 1
# initial_w = np.zeros(tX.shape[1])
# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, batch_size, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = reg_logistic_regression_SGD(y, tX, lambda_, initial_w, batch_size, max_iters, gamma)

In [6]:
# polynomial fit

ratio = 0.9
degree = 10
seed = 6

initial_w = np.zeros(tX.shape[1]*degree)

x_train, y_train, x_val, y_val = split_data(tX, y, ratio, seed)
x_train_poly = build_poly(x_train, degree)
x_val_poly = build_poly(x_val, degree)

In [7]:
# logistic regression GD

# max_iters = 200
# gamma = 1

# weights, loss = logistic_regression_GD(y_train, x_train_poly, initial_w, max_iters, gamma)

# y_train_pred = predict_labels(weights, x_train_poly)
# train_acc = cal_acc(y_train, y_train_pred)
# y_val_pred = predict_labels(weights, x_val_poly)
# val_acc = cal_acc(y_val, y_val_pred)

# print("Train accuracy {ta}, Validation Accuracy {va}".format(ta=train_acc, va=val_acc))

In [8]:
# ridge regression


lambdas = np.logspace(-5, 2, 30)

for lambda_ in lambdas:
    weights, loss = ridge_regression(y_train, x_train_poly, lambda_)

    y_train_pred = predict_labels(weights, x_train_poly)
    train_acc = cal_acc(y_train, y_train_pred)
    y_val_pred = predict_labels(weights, x_val_poly)
    val_acc = cal_acc(y_val, y_val_pred)

    print("Lambda {labd}, Train accuracy {ta}, Validation Accuracy {va}".format(labd=lambda_, ta=train_acc, va=val_acc))

Lambda 1e-05, Train accuracy 0.8151422222222222, Validation Accuracy 0.81436
Lambda 1.7433288221999873e-05, Train accuracy 0.8151422222222222, Validation Accuracy 0.81436
Lambda 3.039195382313195e-05, Train accuracy 0.8151422222222222, Validation Accuracy 0.81432
Lambda 5.2983169062837125e-05, Train accuracy 0.8151511111111112, Validation Accuracy 0.81432
Lambda 9.236708571873866e-05, Train accuracy 0.8151555555555555, Validation Accuracy 0.81432
Lambda 0.00016102620275609394, Train accuracy 0.8151511111111112, Validation Accuracy 0.81436
Lambda 0.0002807216203941176, Train accuracy 0.8151377777777777, Validation Accuracy 0.81436
Lambda 0.0004893900918477494, Train accuracy 0.8151244444444444, Validation Accuracy 0.81436
Lambda 0.0008531678524172815, Train accuracy 0.8151333333333334, Validation Accuracy 0.81436
Lambda 0.0014873521072935117, Train accuracy 0.8151288888888889, Validation Accuracy 0.81436
Lambda 0.002592943797404667, Train accuracy 0.81512, Validation Accuracy 0.81436
La

In [9]:
# best para of ridge regression

ratio = 0.9
degree = 10
lambda_ = 9.236708571873866e-05
seed = 6

x_poly = build_poly(tX, degree)
weights, loss = ridge_regression(y, x_poly, lambda_)

pred = predict_labels(weights, x_poly)
acc = cal_acc(y, pred)

print(acc)

0.816132


## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

# pre-process
bad_data = np.where(tX_test==-999)
bad_col_data = np.where(tX_test[:, -1]==0)

tX_test = (tX_test-col_avg) / col_std
tX_test[bad_data] = 0
tX_test[:, -1][bad_col_data] = 0

In [11]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
# y_pred = predict_labels(weights, tX_test)
# for polynomial fit
x_test_poly = build_poly(tX_test, degree)
y_pred = predict_labels(weights, x_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)